In [1]:
%matplotlib inline

import os
import glob
import librosa
import numpy as np

In [2]:
plain_path = r'audio\c{val}.wav'
variable_path = lambda val: plain_path.format(val=val)
print(variable_path('1'))

audio\c1.wav


In [3]:
labels = {'len', 'xuong', 'trai', 'phai', 'nhay', 'ban','A', 'B'}

In [4]:
# We will use only N occurences per word
N = 25

In [5]:
mfccs = []
true_labels = []
    
for i in range (1,100):        
    sounds = glob.glob(os.path.join(variable_path(i)))
    np.random.shuffle(sounds)
    sounds = sounds[:N]
for l in labels:    
    for s in sounds:
        y, sr = librosa.load(s)
        mfcc = librosa.feature.mfcc(y, sr, n_mfcc=13)
        mfccs.append(mfcc.T)
        true_labels.append(l)
        
mfccs = np.array(mfccs)
true_labels = np.array(true_labels)

C:\Users\HP PC\AppData\Local\Temp\ipykernel_10460\3670627089.py:11: FutureWarning: Pass y=[-0.00152588 -0.00135803 -0.00138855 ... -0.0002594   0.00039673
  0.00111389], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y, sr, n_mfcc=13)


In [11]:
val_percent = 1
n_val = int(val_percent * len(true_labels))

I = np.random.permutation(len(true_labels))
I_val, I_train = I[:n_val], I[n_val:]

In [12]:
from dtw import dtw

def cross_validation(train_indices, val_indices):
    score = 0.0

    for i in val_indices:
        x = mfccs[i]

        dmin, jmin = np.inf, -1
        for j in train_indices:
            y = mfccs[j]
            d, _, _, _ = dtw(x, y, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
            if d < dmin:
                dmin = d
                jmin = j

        score += 1.0 if (true_labels[i] == true_labels[jmin]) else 0.0
        
    return score / len(val_indices)

In [13]:
I_train

array([], dtype=int32)

In [14]:
I_val

array([4, 7, 5, 3, 6, 0, 2, 1])

In [15]:
rec_rate = cross_validation(I_train, I_val)
print('Recognition rate {}%'.format(100. * rec_rate))

Recognition rate 12.5%
